In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import random

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_algorithms.optimizers import SPSA
from scipy.optimize import minimize

# ========================
# 1. H2 Hamiltonian (Pauli list로 직접 정의)
# ========================
pauli_terms = [
    (-1.052373245772859, SparsePauliOp.from_list([("II", 1.0)])),
    (0.39793742484318045, SparsePauliOp.from_list([("ZI", 1.0)])),
    (-0.39793742484318045, SparsePauliOp.from_list([("IZ", 1.0)])),
    (-0.01128010425623538, SparsePauliOp.from_list([("ZZ", 1.0)])),
    (0.18093119978423156, SparsePauliOp.from_list([("XX", 1.0)])),
    (0.18093119978423156, SparsePauliOp.from_list([("YY", 1.0)]))
]

# ========================
# 2. Ansatz (회전 게이트 기반)
# ========================
params = ParameterVector("θ", 4)

def ansatz(params):
    qc = QuantumCircuit(2)
    qc.ry(params[0], 0)
    qc.ry(params[1], 1)
    qc.cx(0, 1)
    qc.ry(params[2], 0)
    qc.ry(params[3], 1)
    return qc

# ========================
# 3. Noisy Estimator 설정
# ========================
backend = AerSimulator()
noise_model = NoiseModel()
# 1-qubit gate noise
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01, 1), ["ry"])
# 2-qubit gate noise
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.05, 2), ["cx"])
# measurement noise
noise_model.add_all_qubit_readout_error(ReadoutError([[0.9, 0.1],[0.05,0.95]]))

estimator = AerEstimator(backend_options={"noise_model": noise_model, "shots": 1024})

# ========================
# 4. Energy evaluation
# ========================
def energy_evaluation(theta_vals):
    qc = ansatz(theta_vals)
    energy = 0.0
    for coeff, pauli in pauli_terms:
        job = estimator.run(circuits=[qc], observables=[pauli])
        result = job.result()
        energy_val = result.values[0]
        energy += coeff * energy_val
    return energy

# ========================
# 5. Rotosolve Optimizer 구현

import numpy as np

class Rotosolve:
    def __init__(self, maxiter=50, tol=1e-8):
        self.maxiter = maxiter
        self.tol = tol
        self.history = []

    @staticmethod
    def _wrap_angle(x):
        return (x + np.pi) % (2 * np.pi) - np.pi

    def optimize(self, init_params, cost_fn):
        params = np.array(init_params, dtype=float)
        for it in range(self.maxiter):
            max_delta = 0.0
            param_indices = list(range(len(params)))
            random.shuffle(param_indices)  # 파라미터 순서 랜덤 섞기

            for i in param_indices:
                theta_i = params[i]

                # 표준 Rotosolve 샘플 3점
                base  = params.copy()                 # θ
                plus  = params.copy(); plus[i] += np.pi/2   # θ + π/2 e_i
                minus = params.copy(); minus[i] -= np.pi/2  # θ - π/2 e_i

                # ---- 배치 평가 시도 ----
                batched_inputs = np.vstack([base, plus, minus])  # (3, D)
                try:
                    vals = cost_fn(batched_inputs)
                    vals = np.asarray(vals, dtype=float).ravel()
                    if vals.shape[0] != 3:
                        raise ValueError("cost_fn batch output must have length 3.")
                    E0, Ep, Em = vals
                except Exception:
                    # 배치 미지원 → 개별 평가로 폴백
                    E0 = float(cost_fn(base))
                    Ep = float(cost_fn(plus))
                    Em = float(cost_fn(minus))
                # -----------------------

                # 닫힌형식 업데이트
                a = np.arctan2(2.0 * E0 - Ep - Em, Ep - Em)
                theta_opt = -np.pi/2 - a
                theta_opt = self._wrap_angle(theta_opt)

                delta = np.abs(self._wrap_angle(theta_opt - theta_i))
                max_delta = max(max_delta, delta)
                params[i] = theta_opt

            cost = float(cost_fn(params))  # 단일점 평가
            self.history.append(cost)

            if (it == 0) or ((it + 1) % 10 == 0):
                print(f"[Rotosolve] Iter {it+1}/{self.maxiter}, Cost = {cost:.6f}")

            if max_delta < self.tol:
                break

        return params, cost

# ========================
# 6. Optimizer 비교 실험
# ========================
optimizers = {
    "Rotosolve": "ROTO",
    "COBYLA": "COBYLA",
    "SPSA": SPSA(maxiter=100),
    "L-BFGS-B": "L-BFGS-B"
}

results = {}
init_params = np.random.rand(4)

for name, opt in optimizers.items():
    print(f"===== Running {name} Optimizer =====")
    start_time = time.time()
    if name == "Rotosolve":
        opt = Rotosolve(maxiter=100)
        params, final_cost = opt.optimize(init_params, energy_evaluation)
        history = opt.history
    elif name in ["COBYLA", "L-BFGS-B"]:
        history = []
        def callback(xk):
            cost = energy_evaluation(xk)
            history.append(cost)
            if len(history) % 10 == 0 or len(history) == 1:
                print(f"[{name}] Iter {len(history)}, Cost = {cost:.6f}")
        res = minimize(fun=energy_evaluation, x0=init_params, method=name, options={"maxiter": 100}, callback=callback)
        params, final_cost = res.x, res.fun
    elif name == "SPSA":
        history = []
        def cost_fn(x):
            val = energy_evaluation(x)
            history.append(val)
            if len(history) % 10 == 0 or len(history) == 1:
                print(f"[SPSA] Iter {len(history)}, Cost = {val:.6f}")
            return val
        res = opt.minimize(fun=cost_fn, x0=init_params)
        params, final_cost = res.x, res.fun
    end_time = time.time()
    exec_time = end_time - start_time
    results[name] = {"final": final_cost, "history": history, "time": exec_time}
    print(f"{name} Final Energy: {final_cost:.6f}, Time = {exec_time:.2f} sec\n")

# ========================
# 7. 시각화 (subplot으로 각각의 optimizer 곡선 표시)
# ========================
n_opt = len(results)
fig, axes = plt.subplots(1, n_opt, figsize=(5*n_opt, 4), sharey=True)

if n_opt == 1:
    axes = [axes]

for ax, (name, data) in zip(axes, results.items()):
    ax.plot(range(1, len(data["history"])+1), data["history"], label=name)
    ax.axhline(-1.137, color="black", linestyle="--", label="Exact Energy")
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Energy (Hartree)")
    ax.set_title(f"{name}\nTime: {data['time']:.2f} sec")
    ax.legend()

plt.suptitle("VQE Optimizer Comparison with Noise (H2)")
plt.tight_layout()
plt.show()

# 최종 정확도 막대그래프
errors = {name: abs(data["final"] - (-1.137)) for name, data in results.items()}
plt.figure(figsize=(6,4))
plt.bar(errors.keys(), errors.values())
plt.ylabel("Absolute Error (Hartree)")
plt.title("Optimizer Accuracy Comparison with Noise")
plt.show()


===== Running Rotosolve Optimizer =====
[Rotosolve] Iter 1/100, Cost = -1.475657
[Rotosolve] Iter 10/100, Cost = -1.307055
[Rotosolve] Iter 20/100, Cost = -1.137283
[Rotosolve] Iter 30/100, Cost = -0.985287
